In [1]:
!pip install transformers datasets accelerate evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 9.6 MB/s eta 0:00:00


In [1]:
from google.colab import drive
drive.mount('/content/drive')

import json
import glob
import os
import pandas as pd
import torch
from sklearn.preprocessing import LabelEncoder
from datasets import Dataset
from transformers import DistilBertTokenizerFast, DistilBertForSequenceClassification
from transformers import Trainer, TrainingArguments
import numpy as np
import evaluate

os.environ["WANDB_DISABLED"] = "true"

Mounted at /content/drive


In [ ]:
df = pd.read_csv("data/responses.csv")

In [5]:
def finetune_by_model(
    df,
    target_col,
    llm_model,
    temperature,
    num_epochs=3,
):

    # Filter by LLM model and remove blank responses
    df_filtered = df[(df['model'] == llm_model) & (df['temperature'] == temperature) & (~df['is_blank'])].copy().reset_index(drop=True)

    if df_filtered.empty:
        raise ValueError(f"No responses found from model '{llm_model}'.")

    # Encode target column
    le = LabelEncoder()
    df_filtered[target_col + "_label"] = le.fit_transform(df_filtered[target_col])
    print(f"{target_col} classes: {le.classes_}")

    dataset = Dataset.from_pandas(df_filtered)

    tokenizer = DistilBertTokenizerFast.from_pretrained("distilbert-base-uncased")
    def tokenize(batch):
        return tokenizer(batch["response"], truncation=True, padding="max_length", max_length=256)

    encoded = dataset.map(tokenize, batched=True)

    num_labels = df_filtered[target_col].nunique()
    model = DistilBertForSequenceClassification.from_pretrained(
        "distilbert-base-uncased",
        num_labels=num_labels
    )

    accuracy = evaluate.load("accuracy")
    def compute_metrics(eval_pred):
        logits, labels = eval_pred
        preds = np.argmax(logits, axis=-1)
        return {"accuracy": accuracy.compute(predictions=preds, references=labels)["accuracy"]}

    training_args = TrainingArguments(
        output_dir="./tmp",
        per_device_train_batch_size=16,
        per_device_eval_batch_size=16,
        num_train_epochs=num_epochs,
        learning_rate=2e-5,
        weight_decay=0.01,
    )

    encoded = encoded.rename_column(target_col + "_label", "labels")
    encoded.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])
    encoded = encoded.train_test_split(test_size=0.2, seed=42)

    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=encoded["train"],
        eval_dataset=encoded["test"],
        tokenizer=tokenizer,
        compute_metrics=compute_metrics,
    )

    trainer.train()

    return trainer, le

In [6]:
targets = ['age', 'gender', 'religion', 'politics']
llm_models = ['grok', 'gemini', 'qwen']
temperatures = [0.95, 1.5]

In [7]:
results = {}

for llm in llm_models:
    if llm not in results:
        results[llm] = {}

    for temp in temperatures:
        if temp not in results[llm]:
            results[llm][temp] = {}

        for target in targets:
            print(f"Training DistilBERT for target '{target}' using LLM '{llm}' at temperature {temp}...")
            try:
                trainer, le = finetune_by_model(
                    df=df,
                    target_col=target,
                    llm_model=llm,
                    temperature=temp,
                    num_epochs=3
                )

                results[llm][temp][target] = {
                    "trainer": trainer,
                    "label_encoder": le
                }

            except ValueError as e:
                print(f"Skipping combination {llm}-{target}-{temp}: {e}")

Training DistilBERT for target 'age' using LLM 'grok' at temperature 0.95...
age classes: ['15' '25' '50']


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

...

Step,Training Loss
500,0.666200
1000,0.590600
1500,0.541600
2000,0.491600


Training DistilBERT for target 'politics' using LLM 'qwen' at temperature 1.5...
politics classes: ['democrat' 'republican']


Map:   0%|          | 0/15997 [00:00<?, ? examples/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
/tmp/ipython-input-2166028279.py:93: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Step,Training Loss
500,0.663500
1000,0.615400
1500,0.582200
2000,0.542900


In [8]:
results

{'grok': {0.95: {'age': {'trainer': <transformers.trainer.Trainer at 0x7f849ce2ac30>,
    'label_encoder': LabelEncoder()},
   'gender': {'trainer': <transformers.trainer.Trainer at 0x7f849a0b30b0>,
    'label_encoder': LabelEncoder()},
   'religion': {'trainer': <transformers.trainer.Trainer at 0x7f850bf8a600>,
    'label_encoder': LabelEncoder()},
   'politics': {'trainer': <transformers.trainer.Trainer at 0x7f850bba3440>,
    'label_encoder': LabelEncoder()}},
  1.5: {'age': {'trainer': <transformers.trainer.Trainer at 0x7f850be24650>,
    'label_encoder': LabelEncoder()},
   'gender': {'trainer': <transformers.trainer.Trainer at 0x7f850be26cc0>,
    'label_encoder': LabelEncoder()},
   'religion': {'trainer': <transformers.trainer.Trainer at 0x7f849e297a40>,
    'label_encoder': LabelEncoder()},
   'politics': {'trainer': <transformers.trainer.Trainer at 0x7f849a1130e0>,
    'label_encoder': LabelEncoder()}}},
 'gemini': {0.95: {'age': {'trainer': <transformers.trainer.Trainer at 0

In [9]:
def save_all(results, base_folder="/content/drive/MyDrive/distilbert_models"):
    import os
    import pickle
    os.makedirs(base_folder, exist_ok=True)

    for llm, temp_dict in results.items():
        for temp, targets in temp_dict.items():
            for target, obj in targets.items():
                # Folder name includes LLM, target, and temperature
                subfolder = os.path.join(base_folder, f"{llm}_{target}_{temp}")
                os.makedirs(subfolder, exist_ok=True)

                trainer = obj["trainer"]
                le = obj["label_encoder"]

                # Save model + tokenizer
                trainer.save_model(subfolder)
                trainer.tokenizer.save_pretrained(subfolder)

                # Save label encoder
                le_path = os.path.join(subfolder, "label_encoder.pkl")
                with open(le_path, "wb") as f:
                    pickle.dump(le, f)

    print("All models saved to Google Drive with temperature accounted for.")

In [10]:
save_all(results)

Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Tr

All models saved to Google Drive with temperature accounted for.
